## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yerbogachen,RU,61.2767,108.0108,35.28,71,98,3.31,overcast clouds
1,1,Vigrestad,NO,58.5667,5.7000,59.59,96,100,21.61,light rain
2,2,Albany,US,42.6001,-73.9662,46.54,94,6,3.78,clear sky
3,3,Mataura,NZ,-46.1927,168.8643,49.48,91,96,3.02,overcast clouds
4,4,Airai,TL,-8.9266,125.4092,61.93,77,22,0.51,few clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Boke,GN,10.9409,-14.2967,80.06,82,100,0.25,overcast clouds
7,7,Sinnamary,GF,5.3833,-52.9500,83.26,73,58,9.62,broken clouds
14,14,Avarua,CK,-21.2078,-159.7750,75.25,83,41,8.05,scattered clouds
17,17,Port Keats,AU,-14.2500,129.5500,80.26,85,68,8.50,broken clouds
20,20,Maputo,MZ,-25.9653,32.5892,80.28,69,75,8.01,broken clouds
30,30,Santa Lucia,ES,27.9117,-15.5407,75.76,71,20,25.32,few clouds
33,33,Yanai,JP,33.9667,132.1167,77.02,80,0,5.93,clear sky
35,35,Bac Lieu,VN,9.2850,105.7244,81.55,81,94,7.54,overcast clouds
41,41,Barao De Melgaco,BR,-16.1944,-55.9675,84.72,39,22,9.37,few clouds
45,45,Cap Malheureux,MU,-19.9842,57.6142,76.46,64,74,10.00,broken clouds


In [27]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Boke,GN,80.06,overcast clouds,10.9409,-14.2967,NaN
7,Sinnamary,GF,83.26,broken clouds,5.3833,-52.9500,NaN
14,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,NaN
17,Port Keats,AU,80.26,broken clouds,-14.2500,129.5500,NaN
20,Maputo,MZ,80.28,broken clouds,-25.9653,32.5892,NaN
30,Santa Lucia,ES,75.76,few clouds,27.9117,-15.5407,NaN
33,Yanai,JP,77.02,clear sky,33.9667,132.1167,NaN
35,Bac Lieu,VN,81.55,overcast clouds,9.2850,105.7244,NaN
41,Barao De Melgaco,BR,84.72,few clouds,-16.1944,-55.9675,NaN
45,Cap Malheureux,MU,76.46,broken clouds,-19.9842,57.6142,NaN


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
        # Grab the first hotel from the results and store the name.
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Boke,GN,80.06,overcast clouds,10.9409,-14.2967,Hôtel Rio Nunez - Boké
7,Sinnamary,GF,83.26,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
14,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
17,Port Keats,AU,80.26,broken clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)
20,Maputo,MZ,80.28,broken clouds,-25.9653,32.5892,Pestana Rovuma
...,...,...,...,...,...,...,...
699,Palu,ID,77.14,overcast clouds,-0.8917,119.8707,Hotel Santika Palu
700,Vega De Alatorre,MX,77.86,overcast clouds,20.0333,-96.6333,El Polo Norte
704,Atambua,ID,75.13,few clouds,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
706,Lagunas,PE,75.72,scattered clouds,-5.2269,-75.6753,Hospedaje Eco


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Call the figure to plot the data.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))